In [1]:
import numpy as np
import pandas as pd
import requests
from datetime import datetime
import json

#Set Display options
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width' , 1000)

In [2]:
# Variables
season = "2020-21"

In [3]:
all_players = pd.read_csv (r'./data/' + season + '/Raw_data/all_players.csv', parse_dates= ["news_added"])
all_teams = pd.read_csv (r'./data/' + season + '/Raw_data/all_teams.csv')
all_events = pd.read_csv (r'./data/' + season + '/Raw_data/all_events.csv', parse_dates= ["deadline_time"])
player_types = pd.read_csv(r'./data/' + season + '/Raw_data/player_types.csv')
game_phases = pd.read_csv(r'./data/' + season + '/Raw_data/game_phases.csv')

understat_player = pd.read_csv(r'./data/' + season + '/Raw_data/understat/understat_player.csv')

player_season_history = pd.read_csv(r'./data/' + season + '/Raw_data/player_season_history.csv')
player_gameweek_history = pd.read_csv(r'./data/' + season + '/Raw_data/player_gameweek_history.csv')
#player_gameweek_history = pd.read_csv(r'./data/2019-20/player_past_history/player_gameweek_history_raw.csv')
player_future_fixture = pd.read_csv(r'./data/' + season + '/Raw_data/player_future_fixture.csv', parse_dates = ["kickoff_time"])



In [4]:
understat_player.head()

,id,player_name,games,time,goals,xG,assists,xA,shots,key_passes,yellow_cards,red_cards,position,team_title,npg,npxG,xGChain,xGBuildup
0,453,Son Heung-Min,4,305,6,1.908542,1,1.566852,10,12,0,0,F M,Tottenham,6,1.908542,3.667317,1.263623
1,5555,Dominic Calvert-Lewin,4,343,6,4.412360,0,0.106039,15,1,0,0,F,Everton,6,4.412360,4.070678,1.158628
2,755,Jamie Vardy,4,355,5,3.879281,0,0.162430,7,2,0,0,F,Leicester,1,0.834606,1.213707,0.216671
3,1250,Mohamed Salah,4,360,5,2.968508,0,1.511990,19,14,0,0,F,Liverpool,3,1.446170,3.628054,1.227359
4,468,Callum Wilson,4,356,4,3.759825,1,0.282659,9,4,0,0,F,Newcastle United,2,2.237487,2.755809,0.257033


## Rename columns

In [5]:
player_gameweek_history.head()

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,value,transfers_balance,selected,transfers_in,transfers_out
0,1,2,8,0,False,2020-09-12T11:30:00Z,0.0,3.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,70,0,76656,0,0
1,1,9,19,0,True,2020-09-19T19:00:00Z,2.0,1.0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,69,-16828,68335,995,17823
2,1,23,11,0,False,2020-09-28T19:00:00Z,3.0,1.0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,69,-11451,59793,675,12126
3,1,29,15,0,True,2020-10-04T13:00:00Z,2.0,1.0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,68,-5324,56403,647,5971
4,1,44,12,0,False,2020-10-17T16:30:00Z,1.0,0.0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,68,-4224,53689,616,4840


In [6]:
player_gameweek_history.rename(columns = {"element" : "player_id" , "opponent_team" : "opponent_team_id" , "fixture" : "fixture_id" } , inplace = True)
all_players.rename(columns = {"code" : "player_code", "element_type" : "player_type_id" , "minutes" : "total_minutes" , "id" : "player_id" , "team" : "team_id" , "points_per_game" : "PPG"} , inplace=True)
all_teams.rename(columns = {"id" : "team_id" , "code" : "team_code" , "name" : "team_name" , "short_name" : "team_short_name" , "strength" : "team_strength" , "form" : "team_form"} , inplace = True)
all_events.rename(columns = {"id" : "gameweek_id"} , inplace = True)
player_types.rename(columns = {"id" : "player_type_id" , "singular_name" : "position_name" , "singular_name_short" : "position_name_short"} , inplace = True)
game_phases.rename(columns = {"id" : "phase_id"} , inplace = True)
player_season_history.rename(columns = {"element_code" : "player_code"} , inplace = True)
player_future_fixture.rename(columns = {"id" : "fixture_id" , "team_h" : "home_team_id" , "team_a" : "away_team_id" , "events" : "gameweek_id" , "code" : "fixture_code"} , inplace = True)
#player_gameweek_history.rename(columns = {} , inplace = True)


## Drop columns

In [7]:
all_events.head(5)

,gameweek_id,name,deadline_time,average_entry_score,finished,data_checked,highest_scoring_entry,deadline_time_epoch,deadline_time_game_offset,highest_score,is_previous,is_current,is_next,chip_plays,most_selected,most_transferred_in,top_element,top_element_info,transfers_made,most_captained,most_vice_captained
0,1,Gameweek 1,2020-09-12 10:00:00+00:00,50,True,True,4761681.0,1599904800,0,142.0,False,False,False,"[{'chip_name': 'bboost', 'num_played': 112843}...",259.0,12.0,254.0,"{'id': 254, 'points': 20}",0,4.0,4.0
1,2,Gameweek 2,2020-09-19 10:00:00+00:00,59,True,True,6234344.0,1600509600,0,165.0,False,False,False,"[{'chip_name': 'bboost', 'num_played': 94615},...",259.0,302.0,390.0,"{'id': 390, 'points': 24}",14637421,4.0,254.0
2,3,Gameweek 3,2020-09-26 10:00:00+00:00,43,True,True,6223553.0,1601114400,0,119.0,False,False,False,"[{'chip_name': 'bboost', 'num_played': 86797},...",259.0,164.0,224.0,"{'id': 224, 'points': 17}",20387878,272.0,254.0
3,4,Gameweek 4,2020-10-03 10:00:00+00:00,48,True,True,6563000.0,1601719200,0,160.0,False,False,False,"[{'chip_name': 'bboost', 'num_played': 82267},...",272.0,164.0,37.0,"{'id': 37, 'points': 24}",14570655,254.0,272.0
4,5,Gameweek 5,2020-10-17 10:00:00+00:00,60,True,True,7124486.0,1602928800,0,136.0,True,False,False,"[{'chip_name': 'bboost', 'num_played': 80554},...",164.0,390.0,117.0,"{'id': 117, 'points': 16}",18391649,390.0,254.0


In [8]:
all_events.drop(columns = ["deadline_time_epoch"] , inplace = True)
player_types.drop(columns = ["plural_name" , "plural_name_short"] , inplace = True)
all_players.drop(columns = ["squad_number" , "special" , "transfers_in" , "transfers_out" , "first_name" , "second_name"] , inplace = True)

## Change date columns

In [9]:
all_players['news_added'] = all_players['news_added'].dt.tz_localize(None)
all_events['deadline_time'] = all_events['deadline_time'].dt.tz_localize(None)
player_future_fixture['kickoff_time'] = player_future_fixture['kickoff_time'].dt.tz_localize(None)

player_gameweek_history['kickoff_time'] = pd.to_datetime(player_gameweek_history['kickoff_time'])
player_gameweek_history['kickoff_time'] = player_gameweek_history['kickoff_time'].dt.tz_localize(None)


## Add Columns

In [10]:
player_future_fixture.head()

,fixture_id,fixture_code,home_team_id,team_h_score,away_team_id,team_a_score,event,finished,minutes,provisional_start_time,kickoff_time,event_name,is_home,difficulty,player_id
0,64,2128351,13,NaN,1,NaN,7.0,False,0,False,2020-11-01 16:30:00,Gameweek 7,False,4,1
1,69,2128356,1,NaN,2,NaN,8.0,False,0,False,2020-11-07 15:00:00,Gameweek 8,True,3,1
2,82,2128369,10,NaN,1,NaN,9.0,False,0,False,2020-11-22 16:30:00,Gameweek 9,False,3,1
3,89,2128376,1,NaN,20,NaN,10.0,False,0,False,2020-11-28 15:00:00,Gameweek 10,True,3,1
4,106,2128393,17,NaN,1,NaN,11.0,False,0,False,2020-12-05 15:00:00,Gameweek 11,False,4,1


In [11]:
def opponent_team(row):
    if row['is_home'] == True:
        val = row['away_team_id']
    elif row['is_home'] == False:
        val = row['home_team_id']
    return val

def team(row):
    if row['is_home'] == False:
        val = row['away_team_id']
    elif row['is_home'] == True:
        val = row['home_team_id']
    return val


player_future_fixture['opponent_team_id'] = player_future_fixture.apply(opponent_team,axis=1)
player_future_fixture['team_id'] = player_future_fixture.apply(team,axis=1)


## Calculate New Columns

In [12]:
all_players.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round', 'player_code', 'cost_change_event', 'cost_change_event_fall', 'cost_change_start', 'cost_change_start_fall', 'dreamteam_count', 'player_type_id', 'ep_next', 'ep_this', 'event_points', 'form', 'player_id', 'in_dreamteam', 'news', 'news_added', 'now_cost', 'photo', 'PPG', 'selected_by_percent', 'status', 'team_id', 'team_code', 'total_points', 'transfers_in_event', 'transfers_out_event', 'value_form', 'value_season', 'web_name', 'total_minutes', 'goals_scored', 'assists', 'clean_sheets', 'goals_conceded', 'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat', 'ict_index', 'influence_rank', 'influence_rank_type', 'creativity_rank', 'creativity_rank_type', 'threat_rank', 'threat_rank_type', 'ict_index_rank', 'ict_index_rank_type', 'corners_and_indirect_freekicks_order', 'corners_and_indirect_freekicks_text',
       'direct_freekicks_

In [13]:
all_players.head()

,chance_of_playing_next_round,chance_of_playing_this_round,player_code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,player_type_id,ep_next,ep_this,event_points,form,player_id,in_dreamteam,news,news_added,now_cost,photo,PPG,selected_by_percent,status,team_id,team_code,total_points,...,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text
0,0.0,0.0,37605,0,0,-2,2,0,3,0.0,0.0,0,0.0,1,False,Not included in Arsenal's 25-man Premier Leagu...,2020-10-20 22:30:18.118477,68,37605.jpg,0.0,0.7,n,1,3,0,...,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,588,237,587,236,588,236,589,237,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.0,39476,0,0,-1,1,0,2,0.0,0.0,0,0.0,2,False,Not included in Arsenal's 25-man Premier Leagu...,2020-10-21 10:30:18.546407,49,39476.jpg,0.0,0.1,n,1,3,0,...,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,528,202,518,202,504,193,532,202,NaN,NaN,NaN,NaN,NaN,NaN
2,75.0,100.0,41270,0,0,0,0,0,2,1.1,1.5,1,1.5,3,False,Thigh injury - 75% chance of playing,2020-10-25 21:30:13.186040,55,41270.jpg,1.4,1.1,d,1,3,7,...,0,0,0,0,0,0,51,66.6,9.8,21.0,9.7,154,59,278,87,205,59,219,75,NaN,NaN,4.0,NaN,NaN,NaN
3,100.0,100.0,54694,-1,1,-3,3,0,3,2.0,2.0,2,2.0,4,False,NaN,2020-10-08 09:00:12.305506,117,54694.jpg,3.3,15.8,a,1,3,20,...,0,0,1,0,0,0,73,93.4,102.9,117.0,31.3,97,36,40,30,41,22,49,26,NaN,NaN,6.0,NaN,1.0,NaN
4,100.0,100.0,58822,0,0,-2,2,0,2,0.0,0.0,0,0.0,5,False,NaN,2020-09-23 09:00:14.881983,48,58822.jpg,0.0,0.2,a,1,3,0,...,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,513,197,501,197,480,186,517,197,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
all_players['PP90'] = np.divide(all_players['total_points'] , all_players['total_minutes'])*90 

In [16]:
all_players['PPMM'] = np.divide(all_players['PPG'] , all_players['now_cost']/10)

In [17]:
all_players['VAPM'] = np.divide(all_players['PPG']-2 , all_players['now_cost']/10)

In [19]:
all_players[['web_name' , 'player_type_id', 'total_points' , 'total_minutes' , 'PP90' , 'PPG' , 'PPMM' , 'VAPM']][all_players['total_minutes']>100].sort_values(by='VAPM' , ascending = False).head()

,web_name,player_type_id,total_points,total_minutes,PP90,PPG,PPMM,VAPM
575,Kilman,2,23,270,7.666667,7.7,1.878049,1.390244
143,Chilwell,2,27,270,9.000000,9.0,1.551724,1.206897
39,Hourihane,3,17,137,11.167883,8.5,1.416667,1.083333
278,Bamford,4,49,490,9.000000,8.2,1.389831,1.050847
33,Martínez,1,34,450,6.800000,6.8,1.416667,1.000000


## Save cleaned data

In [20]:
all_players.to_csv (r'./data/' + season + '/Cleaned_data/all_players.csv', index = False, header=True)
all_teams.to_csv (r'./data/' + season + '/Cleaned_data/all_teams.csv', index = False, header = True)
all_events.to_csv (r'./data/' + season + '/Cleaned_data/all_events.csv', index = False, header = True)
player_types.to_csv(r'./data/' + season + '/Cleaned_data/player_types.csv', index = False, header = True)
game_phases.to_csv(r'./data/' + season + '/Cleaned_data/game_phases.csv', index = False, header = True)

player_season_history.to_csv (r'./data/' + season + '/Cleaned_data/player_season_history.csv', index = False, header=True)
player_gameweek_history.to_csv (r'./data/' + season + '/Cleaned_data/player_gameweek_history.csv', index = False, header=True)
player_future_fixture.to_csv (r'./data/' + season + '/Cleaned_data/player_future_fixture.csv', index = False, header=True)